In [ ]:
import pandas as pd 
import numpy as np
import re
import os
import shutil
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("expand.frame_repr",False)
print('import succed!')

In [ ]:
input_path = r'/Users/edz/Documents/yandi_data/AI_data'
input_path_file = r'/patient3.csv'

In [ ]:
df_ym= pd.read_csv(input_path + input_path_file,sep=',',index_col=None,header=None)
print(df_ym.shape)   # 现有库存数据中“影像描述”不为空的数据容量

In [ ]:
# 修改列名
df_ym.columns =  ['来自检查id','送阅时间','病例医院名称','阅片医院名称','病历医生id','病例医生名称','阅片医生id','阅片医生名称',
                '病史','全身病','病例诊断','影像描述','阅片诊断','质控影像描述','质控阅片诊断','主诉、病例检查、处理、处方json串']
df_ym.tail(2) #校验列名修改成功

In [ ]:
df_ym.duplicated(subset='来自检查id').sum()  #判断指定Series的行数据是否存在重复项
# 或者 df_qgy['来自检查id'].duplicated().sum() 

In [ ]:
df_ym['影像描述'] = df_ym['影像描述'].fillna('nan') # 空缺项填充nan
df_ym['影像描述'].isnull().sum()

In [ ]:
# df_ym[df_ym['影像描述'].str.contains('双眼视盘边清色正',re.I)]['影像描述'].value_counts()  

In [ ]:
# 自定义”全角转半角“函数
def strQ2B(ustring):
    """把字符串全角转半角"""
    ss = []
    for s in ustring:   # ustring=['后发障','AION','无法诊断'],s=list中依次取得的各个字符串
        rstring = ""
#         print("每一行值s:{}".format(s)) 
        for uchar in s:  # s='后发障',uchar='后发障'中依次取得的各个字符
            inside_code = ord(uchar)
#                 print(inside_code)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
#             print(rstring)
        ss.append(rstring)
#         print(ss)
    return ss
#     return ''.join(ss)

In [ ]:
#调用函数，实现全角转半角
df_ym['影像描述'] = strQ2B(df_ym['影像描述'])  # ''.join用于合并字符，df_ym.at['影像描述'] = ''.join(strQ2B(str_i))
df_ym.loc[:5,'影像描述']

In [ ]:
# 插入'影描_gai'列
df_ym.insert(df_ym.columns.get_loc('影像描述')+1,'影描_gai',df_ym['影像描述'])

## 去除空格、括号、波浪线等
\s表示空格，b = re.compile('\s')表示匹配空格；df_tw.at[i,'影描_gai'] = re.compile('\s').sub('',str_i)
匹配到空格之后替换为“”（即：去除空格）

In [ ]:
# 去除字符串中的“\n|\t|\r|~|——”
def kong_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
    #     match_str = '(左|右\d期)' 
        if str_i == 'nan':
            pass
        elif len(re.compile('\n|\t|\r|~|——|(|)|,|、|。|\?').findall(str_i)) > 0: # 去除字符串中的“\n|\t|\r|~|——”
#             print(str_i)
            str_i_1 = str_i.replace('\n','').replace('\r','').replace('\t','').replace(',','').replace('。','').replace('、','')    # 去除\n|\t|\r和,以及。   
            str_i_2 = str_i_1.replace('~','-').replace('——','-')  # 改~和————为短中线
            str_i_3 = re.compile('\?').sub('',str_i_2)
            str_i_4 = re.compile(';').sub('',str_i_3)
            df.at[i,'影描_gai'] = str_i_4.replace('(','').replace(')','')  # 去除括号
#             print('__{}'.format(df.at[i,'影描_gai']))
        else:
            pass

    check_df = df[df['影描_gai'].str.contains('\n|\t|\r|~|——|\(|\)|,|、|。|\?|;')]
    if check_df.shape[0] > 0:
        print("剩余{}条数据存在无效标点".format(check_df.shape[0]))
        kong_fuc(df)
    else:
        print('转化结束！！！')
        pass

In [ ]:
kong_fuc(df_ym)    # 调用去除无效标点的函数

In [ ]:
# 去除空格
def kong1_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('\s').findall(str_i)) > 0:
    #         print(str_i)
            df.at[i,'影描_gai'] = re.compile('\s').sub('',str_i)  # 去除字符串首尾的空格
    #         print('__{}'.format(df.at[i,'影描_gai']))
        else:
            pass

    check_df = df[df['影描_gai'].str.contains('\s')]  #校验
    if check_df.shape[0] > 0:
        print("剩余{}条数据存在空格".format(check_df.shape[0]))
        kong1_fuc(df)
    else:
        print('转化结束！！！')
        pass

In [ ]:
kong1_fuc(df_ym)

In [ ]:
########### 校验是否还存在空格
for i in range(df_ym.shape[0]):
    str_i = str(df_ym.at[i,'影描_gai'])
    if str_i == 'nan':
        pass
    elif len(re.compile('\n|\t|\r|~|——|\(|\)|,|、|。|\?|;').findall(str_i)) > 0:
        print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('\?').sub('',str_i)  # 去除字符串首尾的空格
        print('__{}'.format(df_ym.at[i,'影描_gai']))
    else:
        pass

In [ ]:
# 增加“分隔符”的自定义函数
def add_function(x):
    import re
    list_x=[i.start() for i in re.finditer('双|左|右', x)]
#     print(list_x)

    for i in range(len(list_x)):
        x = list(x)
    #     print(list_x[i])
        x.insert(list_x[i]+i,'!')
        x ="".join(x)
#     print(x)
    return x

In [ ]:
#调用函数，增加分隔符
for m in range(df_ym.shape[0]):
    df_ym.at[m,'影描_gai'] = add_function(df_ym.at[m,'影描_gai'])

print(df_ym.loc[:5,'影描_gai'])  #校验

In [ ]:
## 缺少左右眼的——添加“双”标签
# 处理字符串没有“左/右/双”的情形
def lr_shuang_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else:
            str_tempt = re.split('!',str_i)
            str_i_list = list(filter(None,str_tempt))
            str_new_list = []
            for j in range(len(str_i_list)):
                if len(re.compile('(右|左|双)').findall(str_i_list[j])) == 0:
#                     print(str_i_list)
                    str_new_list.append('双' + str_i_list[j])

                else:
                    str_new_list.append(str_i_list[j])
            df.at[i,'影描_gai'] = '!'.join(str_new_list[m] for m in range(len(str_new_list)))
#             print("___{}".format(df.at[i,'影描_gai']))

In [ ]:
lr_shuang_fuc(df_ym)  ## 调用添加“双”标签的函数

In [ ]:
############## 验证“双”补全
for i in range(df_ym.shape[0]):
    str_i = str(df_ym.at[i,'影描_gai'])
    if str_i == 'nan':
        pass
    else:
        str_tempt = re.split('!',str_i)
        str_i_list = list(filter(None,str_tempt))
        str_new_list = []
        for j in range(len(str_i_list)):
            if len(re.compile('(双|左|右)').findall(str_i_list[j])) == 0:
                print(str_i_list[j])

In [ ]:
print("影描_gai——的规格：{}".format(df_ym['影描_gai'].shape[0]))
print("影像描述——的规格：{}".format(df_ym['影像描述'].shape[0]))

# 通用字符串的名称归一化

In [ ]:
# 定义归一化函数——单个标签替换
def single_replace_fuc(patten_x, repl_x,string_x):
    strinfo_0 = re.compile(patten_x, re.I)
    if len(strinfo_0.findall(string_x)) > 0:
#         print(string_x)
        string_x = re.sub(re.compile(patten_x, flags=re.I), repl_x, string_x)
#         print("__{}".format(string_x))
    else:
        pass
    
    return string_x

In [ ]:
####定义”双眼\|右眼\|左眼“转为”双|左|右“的转化函数
#双、左、右眼——名称统一
# 为解决调用一次函数，会出现的“修改不完全的问题”，可以多次调用函数处理遗留的未匹配数据。
def lrs_normal_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else:
            str_i_1 = single_replace_fuc('双眼', '双', str_i)  #str_i_1/2/3均为中间变量,调用函数 single_replace_fuc
            str_i_2 = single_replace_fuc('左眼', '左', str_i_1)
            str_i_3 = single_replace_fuc('右眼', '右', str_i_2)
            df.at[i,'影描_gai'] =  str_i_3

    check_df = df[df['影描_gai'].str.contains('双眼|右眼|左眼')]
    if check_df.shape[0] > 0:
        print("剩余{}条数据包含双眼|右眼|左眼".format(check_df.shape[0]))
        lrs_normal_fuc(df)
    else:
        print('转化结束！！！')
        pass

In [ ]:
lrs_normal_fuc(df_ym)    # 调用“”双眼\|右眼\|左眼“转为”双|左|右“的转化函数

### C/D名称归一化

In [ ]:
####定义C/D拼写错误的转化函数
#C/D拼写错误——名称统一
def normal_cd_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else:
            a  = 'C/D='
            b  = 'C/D'
            strinfo_0 = re.compile('C/D约为')
            strinfo_1 = re.compile('C/D约')
            strinfo_2 = re.compile('C/D', re.I)
            strinfo_3 = re.compile('C/D=', re.I)   # re.I 表示忽略大小写
            strinfo_4 = re.compile('c/d')
            strinfo_5 = re.compile('杯盘比')
            strinfo_6 = re.compile('C:D',re.I)
            strinfo_7 = re.compile('C/D约有')
            strinfo_8 = re.compile('CD比约')
            strinfo_9 = re.compile('CD比')
            strinfo_10 = re.compile('CD=',re.I)
            
            

            if len(strinfo_5.findall(str_i))>0 :  # 均是汉字的匹配不能加re.I,否则识别不出来
#                 print(str_i)
                df.at[i,'影描_gai'] = strinfo_5.sub(a,str_i)
#                 print("~~~~~"+df.at[i,'影描_gai'])
            elif len(strinfo_4.findall(str_i,re.I))>0 :  # 
#                 print(str_i)
                df.at[i,'影描_gai'] = strinfo_4.sub(b,str_i)
#                 print("======"+df.at[i,'影描_gai']) 
            elif len(strinfo_0.findall(str_i))>0 :  # 
                print(str_i)
                df.at[i,'影描_gai'] = strinfo_0.sub(a,str_i)
                print("==="+df.at[i,'影描_gai'])
            elif len(strinfo_7.findall(str_i))>0 :  # 均是汉字的匹配不能加re.I,否则识别不出来
                print(str_i)
                df.at[i,'影描_gai'] = strinfo_7.sub(a,str_i)
                print("~~~~~"+df.at[i,'影描_gai'])
            elif len(strinfo_8.findall(str_i))>0 :  # 均是汉字的匹配不能加re.I,否则识别不出来
                print(str_i)
                df.at[i,'影描_gai'] = strinfo_8.sub(a,str_i)
                print("~~~~~"+df.at[i,'影描_gai'])
            elif len(strinfo_9.findall(str_i))>0 and len(strinfo_8.findall(str_i))==0:  # 均是汉字的匹配不能加re.I,否则识别不出来
                print(str_i)
                df.at[i,'影描_gai'] = strinfo_9.sub(a,str_i)
                print("~~~~~"+df.at[i,'影描_gai'])
            elif len(strinfo_10.findall(str_i))>0 and len(strinfo_3.findall(str_i))==0:  # 均是汉字的匹配不能加re.I,否则识别不出来
                print(str_i)
                df.at[i,'影描_gai'] = strinfo_10.sub(a,str_i)
                print("~~~~~"+df.at[i,'影描_gai'])
            elif len(strinfo_1.findall(str_i))>0 and len(strinfo_0.findall(str_i))==0:  # 排除“Drusen”
    #             print(str_i)
                df.at[i,'影描_gai'] = strinfo_1.sub(a,str_i)
    #             print('***'+df.at[i,'影描_gai'])
            elif len(strinfo_2.findall(str_i))>0 and len(strinfo_3.findall(str_i))==0:
                if len(strinfo_1.findall(str_i))>0:
                    pass
                else:
                    print(str_i)
                    df.at[i,'影描_gai'] = strinfo_2.sub(a,str_i)
                    print('__'+df.at[i,'影描_gai'])
            elif len(strinfo_6.findall(str_i,re.I))>0 :  # 
                print(str_i)
                df.at[i,'影描_gai'] = strinfo_6.sub(a,str_i)
                print("——————"+df.at[i,'影描_gai'])
            else:
                pass

    check_df_jb_0 = df[df['影描_gai'].str.contains('C/D')] 
    check_df_jb = check_df_jb_0[~check_df_jb_0['影描_gai'].str.contains('C/D=')]  #特殊：右视神经病变待排青光眼?——因此有排青标签，易陷入死循环
    if check_df_jb.shape[0] > 0:
        print("剩余{}条数据未转化为".format(check_df_jb.shape[0]))
        print(check_df_jb[['影像描述','影描_gai']])
        normal_cd_fuc(df)
    else:
        print('转化结束！！！')         

In [ ]:
normal_cd_fuc(df_ym)

In [ ]:
df_ym[df_ym['影描_gai'].str.contains('左0\.\d+')]['影描_gai']

In [ ]:
# 处理C/D比书写错误的特殊情形
for i in range(df_ym.shape[0]):
    str_i = str(df_ym.at[i,'影描_gai'])
    if str_i == 'nan':
        pass
    else:
        a  = 'C/D='
        strinfo_0 = re.compile('V/D', re.I)
        strinfo_1 = re.compile('C\.D', re.I)  # 正则匹配C.D需要加\
        strinfo_2 = re.compile('C/C|C/A', re.I)
        strinfo_3 = re.compile('C/', re.I)
        strinfo_4 = re.compile('C/D=', re.I)
        strinfo_5 = re.compile('C/约|C/A约', re.I)
        strinfo_6 = re.compile('C:/D=', re.I)
        strinfo_7 = re.compile('D/D', re.I)
        strinfo_8 = re.compile('/D=|D=', re.I)
        strinfo_9 = re.compile('/D约', re.I)
        if len(strinfo_0.findall(str_i))>0:  # 均是汉字的匹配不能加re.I,否则识别不出来
#             print(str_i)
            df_ym.at[i,'影描_gai'] = strinfo_0.sub(a,str_i)
#             print("~~~~~"+df_ym.at[i,'影描_gai'])
        elif len(strinfo_1.findall(str_i,re.I))>0:
#             print(str_i)
            df_ym.at[i,'影描_gai'] = strinfo_1.sub(a,str_i)
#             print("======"+df_ym.at[i,'影描_gai']) 
        elif len(strinfo_5.findall(str_i))>0 and len(strinfo_4.findall(str_i))==0 :  # 均是汉字的匹配不能加re.I,否则识别不出来
#             print(str_i)
            df_ym.at[i,'影描_gai'] = strinfo_5.sub(a,str_i)
#             print("———————"+df_ym.at[i,'影描_gai'])
        elif len(strinfo_2.findall(str_i))>0 and len(strinfo_4.findall(str_i))==0 :  # 
#             print(str_i)
            df_ym.at[i,'影描_gai'] = strinfo_2.sub(a,str_i)
#             print("==="+df_ym.at[i,'影描_gai'])
        elif len(strinfo_3.findall(str_i))>0 and len(strinfo_4.findall(str_i))==0 :  # 均是汉字的匹配不能加re.I,否则识别不出来
#             print(str_i)
            df_ym.at[i,'影描_gai'] = strinfo_3.sub(a,str_i)
#             print("~~~~~"+df_ym.at[i,'影描_gai'])
        elif len(strinfo_6.findall(str_i))>0 and len(strinfo_4.findall(str_i))==0 :  # 均是汉字的匹配不能加re.I,否则识别不出来
            print(str_i)
            df_ym.at[i,'影描_gai'] = strinfo_6.sub(a,str_i)
            print("~~~~~"+df_ym.at[i,'影描_gai'])
        elif len(strinfo_9.findall(str_i))>0 and len(strinfo_4.findall(str_i))==0:  # 均是汉字的匹配不能加re.I,否则识别不出来
            print(str_i)
            df_ym.at[i,'影描_gai'] = strinfo_9.sub(a,str_i)
            print("~~~~~"+df_ym.at[i,'影描_gai'])
        elif len(strinfo_7.findall(str_i))>0 and len(strinfo_4.findall(str_i))==0:  # 均是汉字的匹配不能加re.I,否则识别不出来
            print(str_i)
            df_ym.at[i,'影描_gai'] = strinfo_7.sub(a,str_i)
            print("~~~~~"+df_ym.at[i,'影描_gai'])
        elif len(strinfo_8.findall(str_i))>0 and len(strinfo_4.findall(str_i))==0:  # 排除“Drusen”
            print(str_i)
            df_ym.at[i,'影描_gai'] = strinfo_8.sub(a,str_i)
            print('***'+df_ym.at[i,'影描_gai'])
        else:
            pass

## C/D的数值标准化

In [ ]:
df_ym[df_ym['影描_gai'].str.contains('V/D',re.I)]['影描_gai'].value_counts()  

In [ ]:
# 将C/D=O.4修正为C/D=0.4——即：将0错写为O的情形
for i in range(df_ym.shape[0]):
    str_i = str(df_ym.at[i,'影描_gai']) 
    if len(re.compile('O\.',re.I).findall(str_i))>0 and len(re.compile('C/D',re.I).findall(str_i))>0:
        print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('O\.',re.I).sub('0.',str_i)
        print("__" + df_ym.at[i,'影描_gai'] )
    else:
        pass

In [ ]:
# 纠正“C/D=.0.5”的情形以及”C/D=约为“
def nornal_num_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if len(re.compile('C/D=\.0').findall(str_i))>0:
#             print(str_i)
            df.at[i,'影描_gai'] = re.compile('C/D=\.0').sub('C/D=0',str_i)
#             print("__" + df.at[i,'影描_gai'])
        elif len(re.compile('C/D=0\.0\.').findall(str_i))>0:  # C/D=0.0.5
            print(str_i)
            df.at[i,'影描_gai'] = re.compile('C/D=0\.0\.').sub('C/D=0.',str_i)
            print("__" + df.at[i,'影描_gai'])
        elif len(re.compile('C/D=约为').findall(str_i))>0:
    #         print(str_i)
            df.at[i,'影描_gai'] = re.compile('C/D=约为').sub('C/D=',str_i)
    #         print("===" + df.at[i,'影描_gai'])
        elif len(re.compile('C/D=比约').findall(str_i))>0:
#             print(str_i)
            df.at[i,'影描_gai'] = re.compile('C/D=比约').sub('C/D=',str_i)
#             print("===" + df.at[i,'影描_gai'])
        elif len(re.compile('C/D=约').findall(str_i))>0:
    #         print(str_i)
            df.at[i,'影描_gai'] = re.compile('C/D=约').sub('C/D=',str_i)
    #         print("*****" + df.at[i,'影描_gai'])
        elif len(re.compile('C/D约').findall(str_i))>0:
    #         print(str_i)
            df.at[i,'影描_gai'] = re.compile('C/D约').sub('C/D=',str_i)
    #         print("*****" + df.at[i,'影描_gai'])
        elif len(re.compile('C/D=有').findall(str_i))>0:
#             print(str_i)
            df.at[i,'影描_gai'] = re.compile('C/D=有').sub('C/D=',str_i)
#             print("*****" + df.at[i,'影描_gai'])
        elif len(re.compile('C/D似').findall(str_i))>0:
            print(str_i)
            df.at[i,'影描_gai'] = re.compile('C/D似').sub('C/D=',str_i)  # C/D似0.9
            print("*****" + df.at[i,'影描_gai'])
        elif len(re.compile('C/D=0/').findall(str_i))>0:
            print(str_i)
            df.at[i,'影描_gai'] = re.compile('C/D=0/').sub('C/D=0.',str_i)
            print("*****" + df.at[i,'影描_gai'])  
        elif len(re.compile('C/D=\d\.\d+--').findall(str_i))>0:  #  处理C/D=0.5--0.6
            print(str_i)            
            df.at[i,'影描_gai'] = re.compile('--').sub('-',str_i)
            print("*****" + df.at[i,'影描_gai'])            
        else:
            pass

In [ ]:
nornal_num_fuc(df_ym)

In [ ]:
df_ym[df_ym['影描_gai'].str.contains('双眼晶体混浊')]['影描_gai']

### 纠正以下情形：
C/D=偏小约0.2 C/D=月0.6余  C/D=-0.6  C/D=-0.8 C/D=-0.3

C/D==0.5 C/D=≈0.55 C/D=≈0.3 C/D=<   C/D=<0.3 C/D=稍大约0.5 C/D=-0.7-0.8 C/D=-0.7

C/D=近1.0 C/D=较大约0.65  C/D=:0.9-1.0  C/D=增大与0.6  左C/D=增大月0.6余

In [ ]:
# 纠正“C/D=.0.5”的情形以及”C/D=约为“
for i in range(df_ym.shape[0]):
    str_i = str(df_ym.at[i,'影描_gai'])
    if len(re.compile('C/D=偏小约').findall(str_i))>0:
#         print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('C/D=偏小约').sub('C/D=',str_i)
#         print("__" + df_ym.at[i,'影描_gai'])
    elif len(re.compile('C/D=月约').findall(str_i))>0:
#         print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('C/D=月约').sub('C/D=',str_i)
#         print("===" + df_ym.at[i,'影描_gai'])
    elif len(re.compile('C/D=月').findall(str_i))>0:
#         print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('C/D=月').sub('C/D=',str_i)
#         print("===" + df_ym.at[i,'影描_gai'])
    elif len(re.compile('C/D=较大约').findall(str_i))>0:
#         print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('C/D=较大约').sub('C/D=',str_i)
#         print("===" + df_ym.at[i,'影描_gai'])
    elif len(re.compile('C/D=稍大约').findall(str_i))>0:
#         print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('C/D=稍大约').sub('C/D=',str_i)
#         print("*****" + df_ym.at[i,'影描_gai'])
    elif len(re.compile('C/D=近').findall(str_i))>0:
#         print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('C/D=近').sub('C/D=',str_i)
#         print("*****" + df_ym.at[i,'影描_gai'])
    elif len(re.compile('C/D=增大与|C/D=增大月').findall(str_i))>0:
#         print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('C/D=增大与|C/D=增大月').sub('C/D=',str_i)
#         print("*****" + df_ym.at[i,'影描_gai'])
    elif len(re.compile('C/D=-|C/D=≈|C/D=<|C/D==|C/D=:').findall(str_i))>0:
#         print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('C/D=-|C/D=≈|C/D=<|C/D==|C/D=:').sub('C/D=',str_i)
#         print("*****" + df_ym.at[i,'影描_gai'])  
    elif len(re.compile('C/D=/').findall(str_i))>0:  # C/D=/6
        print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('C/D=/').sub('C/D=0.',str_i)
        print("*****" + df_ym.at[i,'影描_gai'])  
    elif len(re.compile('C/D=大').findall(str_i))>0:  # C/D=大0.7
#         print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('C/D=大').sub('C/D=',str_i)
#         print("*****" + df_ym.at[i,'影描_gai'])   
    elif len(re.compile('C/D=扩大').findall(str_i))>0:  # C/D=扩大0.7
#         print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('C/D=扩大').sub('C/D=',str_i)
#         print("*****" + df_ym.at[i,'影描_gai']) 
    elif len(re.compile('C/D=哟').findall(str_i))>0:  # C/D=扩大0.7
        print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('C/D=哟').sub('C/D=',str_i)
        print("*****" + df_ym.at[i,'影描_gai']) 

    else:
        pass

In [ ]:
# 寻找类似C/D=03，C/D=.6的情形，均标准化为C/D=0.6  双视盘色好界清C/D=045!右颞上局部RNFLD血管走形大致正常黄斑区大致正常眼底动脉细反
# 查看不同类型的名称
def zero_point_func(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i=='nan':   # 排除str_i为”null“的项
            pass
        else:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
    #         print("______________{}".format(str_i_list))
            value_list = []
            for j in range(len(str_i_list)):
                if len(re.compile('C/D=0',re.I).findall(str_i_list[j]))>0 and len(re.compile('C/D=0\.',re.I).findall(str_i_list[j]))==0:
                # 处理C/D=09
                    print(str_i_list[j])
                    str_i_list[j] = re.compile('0').sub('0.',str_i_list[j],count=1)
                    value_list.append(str_i_list[j])
                    print("___" + str_i_list[j])
                elif len(re.compile('C/D=\.\d+').findall(str_i_list[j]))>0 and len(re.compile('C/D=0\.\d+').findall(str_i_list[j]))==0:  
                    # 处理C/D=.3
                    print(str_i_list[j])
                    str_i_list[j] = re.compile('\.').sub('0.',str_i_list[j],count=1)
                    value_list.append(str_i_list[j])
                    print("====" + str_i_list[j])
                else:
                    value_list.append(str_i_list[j])
            df.at[i,'影描_gai'] = '!'.join( value_list[g] for g in range(len(value_list)))
    #         print("~~~~~~~~~~~~~" + df.at[i,'影描_gai'])

In [ ]:
zero_point_func(df_ym)

In [ ]:
df_ym[df_ym['影描_gai'].str.contains('左\d+')]['影描_gai']

## 处理缺失左、右缺失一级标签的情形
双底欠清晰视乳头色好界清C/D!右0.7盘沿略窄神经纤维层看不清!左0.4未见明显出血渗出

双视盘色淡红C/D!右0.5!左0.6网膜未见出血渗出

右玻璃体混浊!双视乳头色好界清C/D!右0.75!左0.55盘沿形态可无明显RNFLD未见明显出血渗出

In [ ]:
## 黄斑变性AMD
##### 为“双黄斑病变_黄斑变性_AMD!右湿性!左干性 ”的左右眼添加一级标签  右干性!左史性   
### 双黄斑病变!左外伤  双年龄相关性黄斑变!右早期!左晚期 左活动期!右瘢痕期  
def insert_class_function(a, match_x):
#     left, middle, right = a.partition(match_x)   # 使用匹配到的字符进行前中后，3区分隔
    if len(re.compile('C/D',re.I).findall(a)) > 0:
        if len(re.compile(match_x+'0\.\d+').findall(a))>0:
#             print(a)
            index_insert = [i.start() for i in re.finditer(match_x+'0\.\d+',a)]  # 找到匹配字符串的起始位置——索引     
#             print(index_insert[0])
            a = list(a)    # str 无法直接插入，需先转为list
            a.insert(index_insert[0]+1,"C/D=")
            a ="".join(a)   # 将分隔开的字符合并到一起
        elif len(re.compile(match_x+'约为0\.\d+').findall(a))>0:
            print(a)
            index_insert = [i.start() for i in re.finditer(match_x+'约为0\.\d+',a)]  # 找到匹配字符串的起始位置——索引     
#             print(index_insert[0])
            a = list(a)    # str 无法直接插入，需先转为list
            a.insert(index_insert[0]+3,"C/D=")
            a ="".join(a)   # 将分隔开的字符合并到一起
        elif len(re.compile(match_x+'约0\.\d+').findall(a))>0:
            print(a)
            index_insert = [i.start() for i in re.finditer(match_x+'约0\.\d+',a)]  # 找到匹配字符串的起始位置——索引     
#             print(index_insert[0])
            a = list(a)    # str 无法直接插入，需先转为list
            a.insert(index_insert[0]+2,"C/D=")
            a ="".join(a)   # 将分隔开的字符合并到一起
        else:
            pass
        
#     print('__'+a)
    return a

In [ ]:
## 黄斑变性AMD
# 处理“左几期”的情形：
def lr_jiqi_fuc(df, match_str_x):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
    #     match_str = '(左|右\d期)' 
        if str_i == 'nan':
            pass
        elif len(re.compile('C/D',re.I).findall(str_i))== 0:
            pass        
        else:
            df.at[i,'影描_gai'] = insert_class_function(str_i, match_str_x) # 准备匹配检索的字符串——此处为“左2期”或“右1期”等
#             print("__{}".format(df.at[i,'影描_gai']))

#     check_df_jb_0 = df[df['阅_gai'].str.contains('双青光眼')] 
#     check_df_jb = check_df_jb_0[check_df_jb_0['阅_gai'].str.contains(match_str_x+'继发|开角|闭角|新生血管|绝对期')]  #特殊：右视神经病变待排青光眼?——因此有排青标签，易陷入死循环
#     if check_df_jb.shape[0] > 0:
#         print("剩余{}条数据未转化为——青光眼".format(check_df_jb.shape[0]))
#         print(check_df_jb[['阅片诊断','阅_gai']])
# #         insert_class_function(df, match_x)
#     else:
#         print('转化结束！！！')         

In [ ]:
lr_jiqi_fuc(df_ym, '左')   

In [ ]:
lr_jiqi_fuc(df_ym, '右')     

In [ ]:
# 双晶体混浊视盘色淡红界清C/D=!双0.6-0.7!左黄斑区色素紊乱
lr_jiqi_fuc(df_ym, '双')   

### 处理左C/D=0.4!右1.0的情形
C/D=!右07!左约C/D=0.9!左下方RNFL显著薄变 C/D=!右075 C/D=!右06 

C/D=!右1.0!左C/D=0.8 左C/D=0.4!右1.0 C/D=!右1.0!左C/D=0.9 C/D=!右1.0 左C/D=0.9!右1.0

C/D=!右07!左约C/D=0.9! C/D=0.8!右0.6  右C/D=0.5!左1.0!  右C/D=0.5!左1.0  双视乳头色好界清C/D=0.6!左045

C/D=!右C/D=0.8!左1.0  双视盘色好C/D=!右C/D=0.65!左034 C/D=!右C/D=0.6!左05  右C/D=0.9!左1.0 右C/D=0.7!左04!

In [ ]:
# a = '右:0.5!左C/D=0.7!'
# re.compile('右'+'\:\d\.\d+'+'!'+'左'+'C/D=\d\.\d+').findall(a)

In [ ]:
## 处理左C/D=0.4!右1.0的情形 
def insert_class_function01(df, match_x,math_y):
    for i in range(df.shape[0]):
        a = str(df.at[i,'影描_gai'])
        if len(re.compile(match_x+'C/D=\d\.\d+'+'!'+math_y+'\d\.\d+',re.I).findall(a)) > 0:   #'右C/D=0.9!左1.0' C/D=0.45!左1.0
            print(a)
            index_insert = [i.start() for i in re.finditer(math_y+'\d\.\d+',a)]  # 找到匹配字符串的起始位置——索引     
#             print(index_insert[0])
            a = list(a)    # str 无法直接插入，需先转为list
            a.insert(index_insert[0]+1,"C/D=") 
            df.at[i,'影描_gai'] ="".join(a)   # 将分隔开的字符合并到一起
            print('__'+df.at[i,'影描_gai'])    
        elif len(re.compile(match_x+'C/D=\d\.\d+'+'!'+math_y+'\d+',re.I).findall(a)) > 0:   #右C/D=0.65!左034 
            print(a)
            index_insert = [i.start() for i in re.finditer(math_y+'\d+',a)]  # 找到匹配字符串的起始位置——索引     
#             print(index_insert[0])
            a = list(a)    # str 无法直接插入，需先转为list
            a.insert(index_insert[0]+1,"C/D=") 
            df.at[i,'影描_gai'] ="".join(a)   # 将分隔开的字符合并到一起
            print('======'+df.at[i,'影描_gai'])          
        elif len(re.compile(match_x+'\d+'+'!'+math_y+'约C/D=\d\.\d+',re.I).findall(a))>0:   #右07!左约C/D=0.9
            print(a)
            index_insert = [i.start() for i in re.finditer(match_x+'\d+',a)]  # 找到匹配字符串的起始位置——索引     
#             print(index_insert[0])
            a = list(a)    # str 无法直接插入，需先转为list
            a.insert(index_insert[0]+1,"C/D=")
            df.at[i,'影描_gai'] ="".join(a)   # 将分隔开的字符合并到一起
            print('__'+df.at[i,'影描_gai'])
        elif len(re.compile(match_x+'\d+'+'!'+math_y+'C/D=\d\.\d+',re.I).findall(a))>0:   #右07!左C/D=0.9
            print(a)
            index_insert = [i.start() for i in re.finditer(match_x+'\d+',a)]  # 找到匹配字符串的起始位置——索引     
#             print(index_insert[0])
            a = list(a)    # str 无法直接插入，需先转为list
            a.insert(index_insert[0]+1,"C/D=")
            df.at[i,'影描_gai'] ="".join(a)   # 将分隔开的字符合并到一起
            print('__'+df.at[i,'影描_gai'])
        elif len(re.compile(match_x+'\:\d\.\d+'+'!'+math_y+'C/D=\d\.\d+',re.I).findall(a))>0:   # 右:0.5!左C/D=0.7!
            print(a)
            index_insert = [i.start() for i in re.finditer(match_x+'\:\d\.\d+',a)]  # 找到匹配字符串的起始位置——索引  # 匹配三位数杯盘比，   
#             print(index_insert[0])
            a = list(a)    # str 无法直接插入，需先转为list
            a.insert(index_insert[0]+2,"C/D=") 
            df.at[i,'影描_gai'] ="".join(a)   # 将分隔开的字符合并到一起
            print('__'+df.at[i,'影描_gai'])  

        else:
            pass

In [ ]:
insert_class_function01(df_ym, '右','左')

In [ ]:
insert_class_function01(df_ym, '左','右')

In [ ]:
# 处理——双视盘色好界清C/D=0.65!左1.0
def insert_class_function02(df,math_y):
    for i in range(df.shape[0]):
        a = str(df.at[i,'影描_gai'])
        if len(re.compile('C/D=\d\.\d+'+'!'+math_y+'\d\.\d+',re.I).findall(a)) > 0:   #'右C/D=0.9!左1.0' C/D=0.45!左1.0
            print(a)
            index_insert = [i.start() for i in re.finditer(math_y+'\d\.\d+',a)]  # 找到匹配字符串的起始位置——索引     
#             print(index_insert[0])
            a = list(a)    # str 无法直接插入，需先转为list
            a.insert(index_insert[0]+1,"C/D=") 
            df.at[i,'影描_gai'] ="".join(a)   # 将分隔开的字符合并到一起
            print('__'+df.at[i,'影描_gai'])  
        elif len(re.compile('C/D=\d\.\d+'+'!'+math_y+'\d+',re.I).findall(a)) > 0:   #'双视乳头色好界清C/D=0.6!左045
            print(a)
            index_insert = [i.start() for i in re.finditer(math_y+'\d+',a)]  # 找到匹配字符串的起始位置——索引     
#             print(index_insert[0])
            a = list(a)    # str 无法直接插入，需先转为list
            a.insert(index_insert[0]+1,"C/D=") 
            df.at[i,'影描_gai'] ="".join(a)   # 将分隔开的字符合并到一起
            print('__'+df.at[i,'影描_gai'])
        elif len(re.compile('C/D=\d\.\d+.*'+'!'+math_y+'\d\d',re.I).findall(a)) > 0:   #'双视盘色好C/D=!右C/D=0.85下方盘沿丢失网膜出血伴棉絮斑!左09盘沿丢失
            print(a)
            index_insert = [i.start() for i in re.finditer(math_y+'\d\d',a)]  # 找到匹配字符串的起始位置——索引  # 匹配两位数杯盘比，   
#             print(index_insert[0])
            a = list(a)    # str 无法直接插入，需先转为list
            a.insert(index_insert[0]+1,"C/D=") 
            df.at[i,'影描_gai'] ="".join(a)   # 将分隔开的字符合并到一起
            print('__'+df.at[i,'影描_gai'])
        elif len(re.compile('C/D=\d\.\d+.*'+'!'+math_y+'\d\d\d',re.I).findall(a)) > 0:   #'双视盘色好C/D=!右C/D=0.85下方盘沿丢失网膜出血伴棉絮斑!左09盘沿丢失
            print(a)
            index_insert = [i.start() for i in re.finditer(math_y+'\d\d\d',a)]  # 找到匹配字符串的起始位置——索引  # 匹配三位数杯盘比，   
#             print(index_insert[0])
            a = list(a)    # str 无法直接插入，需先转为list
            a.insert(index_insert[0]+1,"C/D=") 
            df.at[i,'影描_gai'] ="".join(a)   # 将分隔开的字符合并到一起
            print('__'+df.at[i,'影描_gai'])             
        else:
            pass

In [ ]:
insert_class_function02(df_ym,'左')

In [ ]:
insert_class_function02(df_ym,'右')

In [ ]:
# 再次调用函数修正C/D=04和C/D=0/7的情形
zero_point_func(df_ym)
nornal_num_fuc(df_ym)

In [ ]:
df_ym[df_ym['影描_gai'].str.contains('左09')]['影描_gai']

### 去除无效'C/D='——打标干扰项

In [ ]:
# 处理”C/D=正常范围“的情形 或者”双底欠清晰视乳头色好界清C/D=!“中的”C/D=“
for i in range(df_ym.shape[0]):
    str_i = str(df_ym.at[i,'影描_gai'])
    str_tempt = re.split('!',str_i)
    str_i_list = list(filter(None,str_tempt))
    value_list = []
#     print(str_i)
    for k in range(len(str_i_list)):
        if len(re.compile('C/D=').findall(str_i_list[k])) > 0 and len(re.compile('C/D=\d\.\d+').findall(str_i_list[k])) ==0:
            str_i_list[k] = re.compile('C/D=').sub('',str_i_list[k])
            value_list.append(str_i_list[k])   
        else:
            value_list.append(str_i_list[k]) 
#     print("__{}".format(value_list))
    
    df_ym.at[i,'影描_gai'] = '!'.join([value_list[g] for g in range(len(value_list))])
#     print("____{}".format(df_ym.at[i,'影描_gai']))

In [ ]:
###############校验”干扰项“全部去除
for i in range(df_ym.shape[0]):
    str_i = str(df_ym.at[i,'影描_gai'])
    str_tempt = re.split('!',str_i)
    str_i_list = list(filter(None,str_tempt))
    value_list = []
#     print(str_i)
    for k in range(len(str_i_list)):
        if len(re.compile('C/D=').findall(str_i_list[k])) > 0 and len(re.compile('C/D=\d\.\d+').findall(str_i_list[k])) ==0:
            print(str_i_list[k])
        else:
            pass

In [ ]:
# 校验还有多少杯盘比缺失一级标签
df_ym[df_ym['影描_gai'].str.contains('双底欠清晰视乳头色好界清C/D=')]['影描_gai']

In [ ]:
# 处理”C/D=0.500.6“的情形—— 中间添加段横线-
def add_duanxian_func(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        str_tempt = re.split('!',str_i)
        str_i_list = list(filter(None,str_tempt))
        value_list = []
    #     print(str_i)
        for k in range(len(str_i_list)):
            if len(re.compile('C/D=\d\.\d+\.\d+').findall(str_i_list[k])) > 0:
                print(str_i_list[k])
                insert_start = [i.start() for i in re.finditer('\.\d+',str_i_list[k])]
                print(insert_start)
                a = list(str_i_list[k])    # str 无法直接插入，需先转为list
                a.insert(insert_start[1]-1,'-')
                str_i_list[k]="".join(a)
                value_list.append(str_i_list[k])
            else:
                value_list.append(str_i_list[k])

        df.at[i,'影描_gai'] = "!".join(value_list[g] for g in range(len(value_list)))
    #     print('__{}'.format(df.at[i,'影描_gai']))

In [ ]:
add_duanxian_func(df_ym)

In [ ]:
############## 校验全部处理——处理”C/D=0.500.6“的情形—— 中间添加段横线-
for i in range(df_ym.shape[0]):
    str_i = str(df_ym.at[i,'影描_gai'])
    str_tempt = re.split('!',str_i)
    str_i_list = list(filter(None,str_tempt))
    value_list = []
#     print(str_i)
    for k in range(len(str_i_list)):
        if len(re.compile('C/D=\d\.\d+\.\d+').findall(str_i_list[k])) > 0:
            print(str_i_list[k])
        else:
            pass

## 添加新标签

In [ ]:
df_ym.insert(df_ym.columns.get_loc('影描_gai')+1,'right_杯盘比C/D',None)
df_ym.insert(df_ym.columns.get_loc('影描_gai')+1,'left_杯盘比C/D',None)

In [ ]:
# 添加脉络膜Coats病的解析标签
def add_new_label_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if len(re.compile('C/D=\d\.\d+').findall(str_i)) > 0:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
            for k in range(len(str_i_list)):
                if (str.find(str_i_list[k],'C/D=')!=-1) and (str.find(str_i_list[k],'双')!=-1):
#                     print(str_i_list[k])
                    if len(re.compile('C/D='+'\d\.\d+'+'-\d\.\d+').findall(str_i_list[k]))>0:
                        index_start = [i.start() for i in re.finditer('C/D='+'\d\.\d+',str_i_list[k])] # 起始索引
                        index_end = [i.end() for i in re.finditer('C/D='+'\d\.\d+'+'-\d\.\d+',str_i_list[k])] # 终止索引 
#                         print(index_start,index_end)
                    else:
                        index_start = [i.start() for i in re.finditer('C/D='+'\d\.\d+',str_i_list[k])] # 起始索引
                        index_end = [i.end() for i in re.finditer('C/D='+'\d\.\d+',str_i_list[k])] # 终止索引                  
#                         print(index_start,index_end)
                    df.at[i,'left_杯盘比C/D'] = str_i_list[k][index_start[0]+4:index_end[0]] # 截取杯盘比
                    df.at[i,'right_杯盘比C/D'] = str_i_list[k][index_start[0]+4:index_end[0]] 
                elif (str.find(str_i_list[k],'C/D=')!=-1) and (str.find(str_i_list[k],'左')!=-1):
#                     print(str_i_list[k])
                    if len(re.compile('C/D='+'\d\.\d+'+'-\d\.\d+').findall(str_i_list[k]))>0:
                        index_start = [i.start() for i in re.finditer('C/D='+'\d\.\d+',str_i_list[k])] # 起始索引
                        index_end = [i.end() for i in re.finditer('C/D='+'\d\.\d+'+'-\d\.\d+',str_i_list[k])] # 终止索引 
#                         print(index_start,index_end)
                    else:
                        index_start = [i.start() for i in re.finditer('C/D='+'\d\.\d+',str_i_list[k])] # 起始索引
                        index_end = [i.end() for i in re.finditer('C/D='+'\d\.\d+',str_i_list[k])] # 终止索引                  
#                         print(index_start,index_end)
                    df.at[i,'left_杯盘比C/D'] = str_i_list[k][index_start[0]+4:index_end[0]] # 截取杯盘比
                elif (str.find(str_i_list[k],'C/D=')!=-1) and (str.find(str_i_list[k],'右')!=-1):
                    if len(re.compile('C/D='+'\d\.\d+'+'-\d\.\d+').findall(str_i_list[k]))>0:
                        index_start = [i.start() for i in re.finditer('C/D='+'\d\.\d+',str_i_list[k])] # 起始索引
                        index_end = [i.end() for i in re.finditer('C/D='+'\d\.\d+'+'-\d\.\d+',str_i_list[k])] # 终止索引 
#                         print(index_start,index_end)
                    else:
                        index_start = [i.start() for i in re.finditer('C/D='+'\d\.\d+',str_i_list[k])] # 起始索引
                        index_end = [i.end() for i in re.finditer('C/D='+'\d\.\d+',str_i_list[k])] # 终止索引                  
#                         print(index_start,index_end)
                    df.at[i,'right_杯盘比C/D'] = str_i_list[k][index_start[0]+4:index_end[0]] 
                else:
                    pass
        else:
            pass

In [ ]:
add_new_label_fuc(df_ym)   # 调用添加新标签的函数

### '影描_gai'字段——含“Coats病”标签的数据容量

In [ ]:
df_ym[df_ym['影描_gai'].str.contains('C/D=\d\.\d+--')]['影描_gai']

In [ ]:
print("'影描_gai'字段——含C/D——数据容量:{}".format(df_ym[df_ym['影描_gai'].str.contains('C/D=\d\.\d+')].shape[0] ))  # 含青光眼的数据容量

In [ ]:
df_ym['left_杯盘比C/D'].value_counts()  # C/D=0.5--

In [ ]:
df_ym['right_杯盘比C/D'].value_counts()

In [ ]:
df_ym[df_ym['left_杯盘比C/D'].isin(['0.500'])][['影像描述','影描_gai']]

In [ ]:
df_ym_jiao = df_ym[['影像描述','影描_gai','left_杯盘比C/D','right_杯盘比C/D']].drop_duplicates(keep='first')  # 去重
## 患者维度
print("'影描_gai'字段去重后——数据容量:{}".format(df_ym_jiao.shape[0]) )  # '影描_gai'字段去重后的数据容量
df_ym_jiao[:5]

In [ ]:
## 患者维度——含白内障
### '影描_gai'字段——含“白内障”标签的数据容量——去重后，唯一标签
df_ym_jiao_notduplicate = df_ym_jiao[df_ym_jiao['影描_gai'].str.contains('C/D=')]   # '影描_gai'字段去重后——含白内障的数据容量
print("'影描_gai'字段去重后_含C/D——数据容量:{}".format(df_ym_jiao_notduplicate.shape[0]))

In [ ]:
df_last_01 = df_ym_jiao.sample(n=5000,replace=False,random_state=None,axis=0)

In [ ]:
df_last_01[df_last_01['影描_gai'].str.contains('C/D=')].shape[0]

In [ ]:
output_path = r'/Users/edz/Documents/yandi_data/导出数据/影像描述'
output_file = r'/杯盘比' + str(df_ym[df_ym['影描_gai'].str.contains('C/D=')].shape[0]) +'_20200331.xlsx'
writer = pd.ExcelWriter(output_path + output_file)
df_last_01.to_excel(writer,sheet_name='含杯盘比_5000',index=None)
df_ym_jiao_notduplicate.to_excel(writer,sheet_name='含杯盘比_去重_'+ str(df_ym_jiao_notduplicate.shape[0]),index=None)
df_ym_jiao.to_excel(writer,sheet_name='影描_gai_去重_'+str(df_ym_jiao.shape[0]),index=None)
writer.save()

In [ ]:
# 输出全量250625的糖网标签
df_ym_quan = df_ym[['来自检查id','影描_gai','left_杯盘比C/D','right_杯盘比C/D']]

In [ ]:
df_ym_quan.shape[0]

In [ ]:
# 导出新增疾病标签
output_path = r'/Users/edz/Documents/yandi_data/AI_data/zn_ai_影像描述标签'
output_file = r'/杯盘比_quan_250625.xlsx'
writer = pd.ExcelWriter(output_path + output_file)
df_ym_quan.to_excel(writer,sheet_name='杯盘比'+str(df_ym_quan.shape[0]),index=None)
writer.save()